<a href="https://colab.research.google.com/github/hammaad2002/OnnxConversion/blob/main/Onnxruntime_if_else_statement_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%%capture
!pip install onnx
!pip install onnxruntime

In [6]:
import torch
import torch.nn as nn
import torch.onnx

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 20)

    def forward(self, x):
        if x.sum() > 0:
            return self.fc1(x)
        else:
            return self.fc2(x)

# Create a model instance
model = MyModel()

# Script the model to cover the whole graph of the model properly
scripted_model = torch.jit.script(model)

# Dummy input
x = torch.randn(1, 10)

# Export the scripted model to ONNX
torch.onnx.export(scripted_model, x, "model.onnx", opset_version=11)

Checking first condition

In [12]:
import onnxruntime as ort
import numpy as np

# Create onnxruntime session and load the model into it
runtime_session = ort.InferenceSession('model.onnx')

# Generate a random input tensor
input_data = np.random.rand(1, 10).astype(np.float32)

# Run the model
output = runtime_session.run(output_names=None, input_feed={"x.1": input_data})

# Print the output
print(output)

[array([[-0.11374693,  0.11781086,  0.2204764 ,  0.0980921 , -0.12393575,
         0.58832127,  0.08381553, -0.06057395, -0.13055053,  0.21798216]],
      dtype=float32)]


In [15]:
output[0].shape

(1, 10)

Checking second condition

In [16]:
import onnxruntime as ort
import numpy as np

# Create onnxruntime session and load the model into it
runtime_session = ort.InferenceSession('model.onnx')

# Generate a random input tensor
input_data = np.random.rand(1, 10).astype(np.float32) * -1

# Run the model
output = runtime_session.run(output_names=None, input_feed={"x.1": input_data})

# Print the output
print(output)

[array([[ 0.33516896, -0.3808381 ,  0.19739884, -0.26054007,  0.3200385 ,
        -0.4683231 ,  0.38515747, -0.039433  , -0.76650953,  0.0456315 ,
         0.21966675, -0.55031645, -0.00447094,  0.31128544, -0.29129508,
        -0.39593104, -0.30141792, -0.3382364 , -0.40622908,  0.0514953 ]],
      dtype=float32)]


In [17]:
output[0].shape

(1, 20)